In [ ]:
from bs4 import BeautifulSoup
import requests
import random, openpyxl
import time

In [ ]:
## Creating excel form for Saving Stream data
excelFile = openpyxl.Workbook()
print(excelFile.sheetnames)
sheet = excelFile.active
sheet.title = "IMDB_Movies_Year_2015"
print(excelFile.sheetnames)
sheet.append(['title_id',
        'title_text',
        'title_type',
        'original_title_text',
        'primary_image_url',
        'caption',
        'release_year',
        'aggregate_rating',
        'vote_count',
        'runtime_seconds',
        'certificate_rating',
        'genres',
        'directors',
        'actors',
        'plot',
        'metascore',
        'released_quarter'])

['Sheet']
['IMDB_Movies_Year_2015']


In [ ]:
import re

def sanitize_for_excel(value):
    """
    Removes characters that are illegal in Excel cells.
    """
    if not isinstance(value, str):
        return value  # Only sanitize strings; leave other types unchanged
    illegal_char_re = re.compile(r'[\000-\010]|[\013-\014]|[\016-\037]')
    return illegal_char_re.sub("", value)


In [ ]:
def fill_data(movie_details):
  movie_details = {key: sanitize_for_excel(value) for key, value in movie_details.items()}
  sheet.append([movie_details['title_id'],
        movie_details['title_text'],
        movie_details['title_type'],
        movie_details['original_title_text'],
        movie_details['primary_image_url'],
        movie_details['caption'],
        movie_details['release_year'],
        movie_details['aggregate_rating'],
        movie_details['vote_count'],
        movie_details['runtime_seconds'],
        movie_details['certificate_rating'],
        movie_details['genres'],
        movie_details['directors'],
        movie_details['actors'],
        movie_details['plot'],
        movie_details['metascore'],
        'January-March'])
def make_request(url, headers, params):
    response = requests.get(url, headers=headers, params=params)
    print(response.response())
    return response.json()

def fetch_directors_and_actors(title_id):
    # API endpoint for fetching directors and actors might be different
    details_url = "https://caching.graphql.imdb.com/"  # Adjust if necessary

    # Preparing the variables section with the dynamic title_id
    d_q_dict ={"operationName":"Title_Summary_Prompt_From_Base","variables":"{\"id\":\"tt0237572\",\"locale\":\"en-US\",\"location\":{\"latLong\":{\"lat\":\"9.11\",\"long\":\"-8.44\"}},\"promotedProvider\":null,\"providerId\":null}","extensions":"{\"persistedQuery\":{\"sha256Hash\":\"28fdb27482f0852bc70811f36a5fab72e9afd1589b37c5e1aaecf438d418f6c7\",\"version\":1}}"}
    d_variables_dict = json.loads(d_q_dict['variables'])
    d_variables_dict['id'] = title_id
    d_q_dict['variables'] = json.dumps(d_variables_dict, separators=(',', ':'))

    # Making the request
    response = make_request(details_url, headers, d_q_dict)
    data = response.get('data', {}).get('title', {})
    #print(data)

    # Extracting directors
    directors = [credit.get('name', {}).get('nameText', {}).get('text', '')
                 for director in data.get('principalDirectors', [])
                 for credit in director.get('credits', [])]
    #print('directors;-', directors)
    # Extracting actors
    actors = [credit.get('name', {}).get('nameText', {}).get('text', '')
              for cast in data.get('principalCast', [])
              for credit in cast.get('credits', [])]
    #print('actors;-', actors)

    return directors, actors


def extract_movie_details(edge):
    node = edge.get('node', {}).get('title', {}) if edge.get('node') else {}

    # Prepare default values upfront to avoid repeated type checks
    default_float = 0.0
    default_int = 0
    default_str = ''

    movie_details = {
        'title_id': node.get('id', default_str),
        'title_text': node.get('titleText', {}).get('text', default_str) if node.get('titleText') and node.get('titleText').get('text') is not None else default_str,
        'title_type': node.get('titleType', {}).get('text', default_str) if node.get('titleType') and node.get('titleType').get('text') is not None else default_str,
        'original_title_text': node.get('originalTitleText', {}).get('text', default_str) if node.get('originalTitleText') and node.get('originalTitleText').get('text') is not None else default_str,
        'primary_image_url': node.get('primaryImage', {}).get('url', default_str) if node.get('primaryImage') and node.get('primaryImage').get('url') is not None else default_str,
        'caption': node.get('primaryImage', {}).get('caption', {}).get('plainText', default_str) if node.get('primaryImage') and node.get('primaryImage').get('caption') and node.get('primaryImage').get('caption').get('plainText') is not None else default_str,
        'release_year': str(node.get('releaseYear', {}).get('year', default_str)) if node.get('releaseYear') and node.get('releaseYear').get('year') is not None else default_str,
        'aggregate_rating': float(node.get('ratingsSummary', {}).get('aggregateRating', default_float)) if node.get('ratingsSummary') and node.get('ratingsSummary').get('aggregateRating') is not None else default_float,
        'vote_count': int(node.get('ratingsSummary', {}).get('voteCount', default_int)) if node.get('ratingsSummary') and node.get('ratingsSummary').get('voteCount') is not None else default_int,
        'runtime_seconds': int(node.get('runtime', {}).get('seconds', default_int)) if node.get('runtime') and node.get('runtime').get('seconds') is not None else default_int,
        'certificate_rating': node.get('certificate', {}).get('rating', default_str) if node.get('certificate') and node.get('certificate').get('rating') is not None else default_str,
        'genres': '',
        'directors': '',
        'actors': '',
        'plot': node.get('plot', {}).get('plotText', {}).get('plainText', default_str) if node.get('plot') and node.get('plot').get('plotText') and node.get('plot').get('plotText').get('plainText') is not None else default_str,
        'metascore': int(node.get('metacritic', {}).get('metascore', {}).get('score', default_int)) if node.get('metacritic') and node.get('metacritic').get('metascore') and node.get('metacritic').get('metascore').get('score') is not None else default_int
    }
    title_genres = node.get('titleGenres', {})  # Attempt to get 'titleGenres'
    genres_list = title_genres.get('genres') if title_genres else []  # Safely get 'genres', ensuring title_genres is not None
    genres = [genre.get('genre', {}).get('text', default_str) for genre in genres_list if genre and genre.get('genre')]
    movie_details['genres'] = '|'.join(genres)

    # Fetch directors and actors, ensuring the title_id is not empty
    if movie_details['title_id']:
        directors, actors = fetch_directors_and_actors(movie_details['title_id'])
        movie_details['directors'] = '|'.join(directors) if directors else default_str
        movie_details['actors'] = '|'.join(actors) if actors else default_str

    return movie_details



In [ ]:
import requests
import json

def make_request(url, headers, query_dict):
    response = requests.get(url, headers=headers, params=query_dict)
    return response.json()

url = "https://caching.graphql.imdb.com/"

query_dict = {"operationName":"AdvancedTitleSearch","variables":"{\"first\":350,\"locale\":\"en-US\",\"releaseDateConstraint\":{\"releaseDateRange\":{\"end\":\"2015-03-31\",\"start\":\"2015-01-01\"}},\"sortBy\":\"POPULARITY\",\"sortOrder\":\"ASC\",\"titleTypeConstraint\":{\"anyTitleTypeIds\":[\"movie\"]}}","extensions":"{\"persistedQuery\":{\"sha256Hash\":\"28be9832bab4e1acaccc405bc13dc9976ce0bfb2a9149a189a5833879d724852\",\"version\":1}}"}
#query_dict = {"operationName":"AdvancedTitleSearch","variables":"{\"after\":\"eyJlc1Rva2VuIjpbIjEyODQ2IiwiMTI4NDYiLCJ0dDQyNzkwMTIiXSwiZmlsdGVyIjoie1wiY29uc3RyYWludHNcIjp7XCJyZWxlYXNlRGF0ZUNvbnN0cmFpbnRcIjp7XCJyZWxlYXNlRGF0ZVJhbmdlXCI6e1wiZW5kXCI6XCIyMDE1LTAzLTMxXCIsXCJzdGFydFwiOlwiMjAxNS0wMS0wMVwifX19LFwibGFuZ3VhZ2VcIjpcImVuLVVTXCIsXCJzb3J0XCI6e1wic29ydEJ5XCI6XCJQT1BVTEFSSVRZXCIsXCJzb3J0T3JkZXJcIjpcIkFTQ1wifSxcInJlc3VsdEluZGV4XCI6OTl9In0=\",\"first\":50,\"locale\":\"en-US\",\"releaseDateConstraint\":{\"releaseDateRange\":{\"end\":\"2015-03-31\",\"start\":\"2015-01-01\"}},\"sortBy\":\"POPULARITY\",\"sortOrder\":\"ASC\"}","extensions":"{\"persistedQuery\":{\"sha256Hash\":\"28be9832bab4e1acaccc405bc13dc9976ce0bfb2a9149a189a5833879d724852\",\"version\":1}}"}
headers = {
    "accept": "application/graphql+json, application/json",
    "accept-language": "en-US,en;q=0.9",
    "content-type": "application/json",
    "cookie": "session-id=135-9490825-3490238; session-id-time=2082787201l; ubid-main=130-8744775-9760067; _cc_id=2903319d3c7c9a713151486d73e4a339; uu=eyJpZCI6InV1ZTAxMmNmODhmNjk1NGY2N2IzZWIiLCJwcmVmZXJlbmNlcyI6eyJmaW5kX2luY2x1ZGVfYWR1bHQiOmZhbHNlfX0=; _gcl_au=1.1.299470935.1712553986; session-token=qJNix/39gy2KnaptEScptb8N9HEVTtML9PKdWJ0DGcIuPBtbM7OQWwY+4gin3+MFbqAxY/yQGBtfDcO6oKsu4uC0ollkCz+ewI/ufhzVvq5NaNueToFoHmcWjLupTY+sOd2INOit23qTPB9UGOJL7PjnzvMPvEKdAGI5Yc/zWb4JeOvDvZTw7XKtEySY+EKnrjMKvPGR9auRcaR9HRGXn8iyrX23+03HxhPDNtWxEX3AtYtEDbAlIIbSTnfV8yvgKPYvvuzaSEVXLBBFpaEYI/nwYwb494I8hVriGbSXNgk84RuKDW+OfPySO+80+mFC7FSyRwW0tgnFBu8TDv8WYltxQ9fMNFXr; ad-oo=0; ci=e30; __gads=ID=f39e62066a779a92:T=1712259068:RT=1713487490:S=ALNI_MZ6CQOJ9XCB5ZqQcJdVqeZ0CTanwA; __eoi=ID=66ff0bb1aed96d28:T=1712259068:RT=1713487490:S=AA-AfjaOBAvd5VNhhSRLmJt_zBmV",
    "origin": "https://www.imdb.com",
    "referer": "https://www.imdb.com/",
    "sec-ch-ua": "'Google Chrome';v='123', 'Not:A-Brand';v='8', 'Chromium';v='123'",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "'macOS'",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "x-amzn-sessionid": "135-9490825-3490238",
    "x-imdb-client-name": "imdb-web-next-localized",
    "x-imdb-client-rid": "WYTS0YBEND5HE0VFWBC9",
    "x-imdb-user-country": "US",
    "x-imdb-user-language": "en-US",
    "x-imdb-weblab-treatment-overrides": '{"IMDB_NAV_PRO_FLY_OUT_Q1_REFRESH_848923":"T1"}'
}


# Initialize hasNextPage to True to enter the loop
hasNextPage = True
i = 1
while hasNextPage:
    # Make the API request
    data = make_request(url, headers, query_dict)
    #print(data)
    #print('edges:- ',data['data']['advancedTitleSearch']['edges'][1])
    #break
    for m in data['data']['advancedTitleSearch']['edges']:
      movie_details = extract_movie_details(m)
    #print(movie_details)
      fill_data(movie_details)
      print("i:- ", i)
      i += 1
    # Process your data here
    # Example: print(data)

    # Check if there's a next page
    hasNextPage = data['data']['advancedTitleSearch']['pageInfo']['hasNextPage']
    #print('hasNextPage:-', hasNextPage)
    #i += 1
    if hasNextPage:
        endCursor = data['data']['advancedTitleSearch']['pageInfo']['endCursor']
        variables_dict = json.loads(query_dict['variables'])
        variables_dict['after'] = endCursor
        query_dict['variables'] = json.dumps(variables_dict, separators=(',', ':'))
    time.sleep(30)

i:-  1
i:-  2
i:-  3
i:-  4
i:-  5
i:-  6
i:-  7
i:-  8
i:-  9
i:-  10
i:-  11
i:-  12
i:-  13
i:-  14
i:-  15
i:-  16
i:-  17
i:-  18
i:-  19
i:-  20
i:-  21
i:-  22
i:-  23
i:-  24
i:-  25
i:-  26
i:-  27
i:-  28
i:-  29
i:-  30
i:-  31
i:-  32
i:-  33
i:-  34
i:-  35
i:-  36
i:-  37
i:-  38
i:-  39
i:-  40
i:-  41
i:-  42
i:-  43
i:-  44
i:-  45
i:-  46
i:-  47
i:-  48
i:-  49
i:-  50
i:-  51
i:-  52
i:-  53
i:-  54
i:-  55
i:-  56
i:-  57
i:-  58
i:-  59
i:-  60
i:-  61
i:-  62
i:-  63
i:-  64
i:-  65
i:-  66
i:-  67
i:-  68
i:-  69
i:-  70
i:-  71
i:-  72
i:-  73
i:-  74
i:-  75
i:-  76
i:-  77
i:-  78
i:-  79
i:-  80
i:-  81
i:-  82
i:-  83
i:-  84
i:-  85
i:-  86
i:-  87
i:-  88
i:-  89
i:-  90
i:-  91
i:-  92
i:-  93
i:-  94
i:-  95
i:-  96
i:-  97
i:-  98
i:-  99
i:-  100
i:-  101
i:-  102
i:-  103
i:-  104
i:-  105
i:-  106
i:-  107
i:-  108
i:-  109
i:-  110
i:-  111
i:-  112
i:-  113
i:-  114
i:-  115
i:-  116
i:-  117
i:-  118
i:-  119
i:-  120
i:-  121
i:-  122
i:-  123
i

In [ ]:
#Saving Data File for Year Wise Data
excelFile.save('imdb_year_2015.xlsx')